In [1]:
import json
import numpy as np
from tqdm import tqdm

In [2]:
data_train = []
with open('PIZZA_train.json', 'r') as f:
    for line in f:
        data_train.append(json.loads(line))

print(len(data_train))

2456446


In [3]:
with open('PIZZA_dev.json', 'r') as file:
    data_dev = json.load(file)

# Print the loaded data
print(len(data_dev))

348


In [4]:
s=np.load('processed_sentences_no_conct_train.npy')

In [5]:
def extract_text_train(data):
    return data['train.SRC'], data['train.EXR'], data['train.TOP'], data['train.TOP-DECOUPLED']

In [6]:
def extract_text_dev(data):
    return data['dev.SRC'], data['dev.EXR'], data['dev.TOP'], data['dev.PCFG_ERR']

In [7]:
_, explanations_dev, topics_dev, decoupled_topics_dev = map(
    np.array, zip(*map(extract_text_dev, data_dev)))

In [8]:
_, explanations_train, topics_train, decoupled_topics_train = map(
    np.array, zip(*map(extract_text_train, data_train)))

In [9]:
entities_dev = set([word[1:]
               for t in topics_dev for word in t.split() if word.isupper()])

entities_train = set([word[1:]
               for t in topics_train for word in t.split() if word.isupper()])

full_entities = entities_dev | entities_train

enitities_exclude_not = full_entities - {'NOT'}

negate_mapping = {}
for entity in full_entities:
    if entity != 'NOT':
        negate_mapping[entity] = "NOT_" + entity


In [10]:
negate_mapping

{'SIZE': 'NOT_SIZE',
 'QUANTITY': 'NOT_QUANTITY',
 'ORDER': 'NOT_ORDER',
 'VOLUME': 'NOT_VOLUME',
 'CONTAINERTYPE': 'NOT_CONTAINERTYPE',
 'COMPLEX_TOPPING': 'NOT_COMPLEX_TOPPING',
 'STYLE': 'NOT_STYLE',
 'DRINKORDER': 'NOT_DRINKORDER',
 'DRINKTYPE': 'NOT_DRINKTYPE',
 'NUMBER': 'NOT_NUMBER',
 'TOPPING': 'NOT_TOPPING',
 'PIZZAORDER': 'NOT_PIZZAORDER'}

In [11]:
def negate_topic(topic):
    not_array = topic.split('NOT')
    not_array = not_array[1:]
    final_array = []
    for element in not_array:
        element = element.strip()
        open_brackets = 1
        result = ""
        index = 0
        while open_brackets > 0:
            if element[index] == '(':
                open_brackets += 1
            elif element[index] == ')':
                open_brackets -= 1
            if open_brackets > 0:
                result += element[index]
            index += 1

        final_array.append(result.strip())
    return final_array


def apply_negation(topic):
    array = negate_topic(topic)
    for el in array:
        original_el = el
        for entity in enitities_exclude_not:
            el = el.replace(entity, negate_mapping[entity])
            
        topic = topic.replace(original_el, el)
    return topic

In [12]:
negated_topics_train = np.vectorize(apply_negation)(topics_train)

In [13]:
negated_topics_dev = np.vectorize(apply_negation)(topics_dev)

In [14]:
final_entities = full_entities.copy()
final_entities.remove('NOT')
final_entities.remove('COMPLEX_TOPPING')
final_entities.remove('PIZZAORDER')
final_entities.remove('DRINKORDER')
final_entities.remove('ORDER')
print(final_entities)

{'SIZE', 'DRINKTYPE', 'QUANTITY', 'NUMBER', 'VOLUME', 'CONTAINERTYPE', 'TOPPING', 'STYLE'}


In [15]:
negate_mapping = {}
for entity in final_entities:
    if entity != 'NOT':        negate_mapping[entity] = "NOT_" + entity

print(negate_mapping)

{'SIZE': 'NOT_SIZE', 'DRINKTYPE': 'NOT_DRINKTYPE', 'QUANTITY': 'NOT_QUANTITY', 'NUMBER': 'NOT_NUMBER', 'VOLUME': 'NOT_VOLUME', 'CONTAINERTYPE': 'NOT_CONTAINERTYPE', 'TOPPING': 'NOT_TOPPING', 'STYLE': 'NOT_STYLE'}


In [16]:
final_negated_entities = final_entities | set(negate_mapping.values())

In [17]:
print(len(final_negated_entities))

16


In [18]:
np.save("final_negated_entites.npy",np.array(list(final_negated_entities)))

In [22]:
def get_entity(topic,mapping):
    # print(mapping)
    words_array=topic.split()
    result=[]
    current_entity='O'
    first=False
    for word in words_array:
        if word.startswith("("):
            if word[1:] in mapping:
                current_entity=word[1:]
                first=True
            else:
                current_entity='O'
        elif word.startswith(")"):
            current_entity='O'
        else:
            if current_entity=='O':
                result.append(current_entity)
            else:
                if first:
                    result.append('B-'+current_entity)
                    first=False
                else:
                    result.append('I-'+current_entity)    
    return result
        

number=1205
print(s[number])
print(negated_topics_train[number])
mp=get_entity(negated_topics_train[number],final_negated_entities)
print(mp)
# print(tokens[500])

party - size pizza with banana peppers and peperonni and with yellow peppers
(ORDER (PIZZAORDER (SIZE party - size ) pizza with (TOPPING banana peppers ) and (TOPPING peperonni ) and with (TOPPING yellow peppers ) ) )
['B-SIZE', 'I-SIZE', 'I-SIZE', 'O', 'O', 'B-TOPPING', 'I-TOPPING', 'O', 'B-TOPPING', 'O', 'O', 'B-TOPPING', 'I-TOPPING']


In [23]:
words_to_entities_train = [get_entity(topic, final_negated_entities) for topic in 
                           tqdm(negated_topics_train,desc="train")]
words_to_entities_dev = [get_entity(topic, final_negated_entities) for topic in 
                            tqdm(negated_topics_dev,desc="dev")]

train:   0%|          | 0/2456446 [00:00<?, ?it/s]

dev: 100%|██████████| 348/348 [00:00<00:00, 103000.34it/s]


In [29]:
number=205
print(negated_topics_dev[number])
print(words_to_entities_dev[number])

(ORDER my order today is for (PIZZAORDER (NUMBER a ) (SIZE medium ) pizza i like the (STYLE thin crust ) and i will try the (TOPPING tuna ) but no (NOT (NOT_TOPPING pineapple ) ) ) thanks a lot )
['O', 'O', 'O', 'O', 'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'O', 'O', 'B-STYLE', 'I-STYLE', 'O', 'O', 'O', 'O', 'O', 'B-TOPPING', 'O', 'O', 'B-NOT_TOPPING', 'O', 'O', 'O']


In [23]:
np.save("words_to_entities_train.npy", np.array(words_to_entities_train,dtype=object))
np.save("words_to_entities_dev.npy", np.array(words_to_entities_dev,dtype=object))